In [1]:
import sys, os
sys.path.append(os.path.dirname(os.getcwd())) # Include ../SSD in path

# Introduction to SSD

This code is much more complex than your prior assignment, and we recommend you to spend some time getting familiar with the code structure.
The "complex" code structure is made to simplify aspects of deep learning experimentation, and help you structure the usual "sphagetti" deep learning code.


All scripts in this code requires a configuration file. To **start training**, you can type:
```
python train.py configs/ssd300.py
```

## Configuration files
The key difference from previous starter codes is the use of configuration files. This enables us to change small parts of the experiment without changing hard-coded values (e.g. learning rate in previous assignments).

If you take a look in [`configs/ssd300.py`](../configs/ssd300.py) you will notice a large set of objects describing model architecture (`backbone` and `model`), the optimizer, dataset, data loading, and hyperparameters.

To load the config we can write the following:

In [2]:
from ssd.utils import load_config
cfg = load_config("../configs/ssd300.py")

Saving SSD outputs to: outputs/


`cfg` supports access syntax, where all objects in `configs/ssd300.py` are accessible via their attribute name.



In [3]:
print(cfg.model)

{'feature_extractor': '${backbone}', 'anchors': '${anchors}', 'loss_objective': '${loss_objective}', 'num_classes': 11, '_target_': <class 'ssd.modeling.ssd.SSD300'>}


If we print `cfg.model`, notice that it returns a dictionary and not the model object itself (which is `SSD300` from [`ssd/modeling/ssd.py`](../ssd/modeling/ssd.py)). This is because the model is defined "lazily" (wrapped with a `LazyCall`).

To create the model, we have to instantiate it:

In [4]:
from tops.config import instantiate
model = instantiate(cfg.model)
print(model)

SSD300(
  (feature_extractor): BasicModel()
  (loss_func): SSDMultiboxLoss(
    (sl1_loss): SmoothL1Loss()
  )
  (regression_heads): ModuleList(
    (0): Conv2d(128, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Conv2d(256, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2-3): 2 x Conv2d(128, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4-5): 2 x Conv2d(64, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (classification_heads): ModuleList(
    (0): Conv2d(128, 44, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Conv2d(256, 66, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2-3): 2 x Conv2d(128, 66, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4-5): 2 x Conv2d(64, 44, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
)


Another example, we can load the first batch of the dataset and run a forward pass with the model:

(Task4a needs to be implemented in order for this to run)

In [5]:
dataloader_train = instantiate(cfg.data_train.dataloader)
print(type(dataloader_train))
batch = next(iter(dataloader_train))
for key, item in batch.items():
    print(key, "has the shape:", item.shape)
gpu_transform = instantiate(cfg.data_train.gpu_transform)
batch = gpu_transform(batch)
bbox_delta, confidences = model(batch["image"])
print(f"The model predicted anchors with  bbox delta: {bbox_delta.shape} and confidences: {confidences.shape}")

FileExistsError: [Errno 17] File exists: 'data'

You might ask yourself, why? At first sight, this seems very complicated rather than plain-old  hard coded values.

The reason is easy manipulation of experiments. If I want to run the same experiment, but with a different batch size, I can change it with the following:

In [ ]:
# Lets print the batch size of the original data loader:
print("Original batch size:", dataloader_train.batch_size)
cfg.train.batch_size = 2 # Setting the batch size to 2
dataloader_train = instantiate(cfg.data_train.dataloader)
print("New batch size:", dataloader_train.batch_size)

Another reason is **configuration inheritance**.  This allows us to change cirtain aspects of the config without defining the config from scratch.

Take a look in [`configs/change_lr.py`](../configs/change_lr.py) and notice that we inherit from the original config file.
The only changes done are to the lr.

In [ ]:
cfg = load_config("../configs/change_lr.py")
model = instantiate(cfg.model)
print(model)

# Useful commands:


#### Training and evaluation
To start training:
```
python train.py  configs/ssd300.py
```

To only run evaluation:
```
python train.py  configs/ssd300.py --evaluate-only
```

#### Demo.py

For MNIST:
```
python demo.py configs/ssd300.py demo/mnist demo/mnist_output
```


#### Runtime analysis:
```
python3 runtime_analysis.py configs/ssd300.py
```